Primero importamos pandas y cargamos nuestro dataset desde una ubicación que pueda leerse con read_csv:

In [61]:
import pandas as pd

laptops_dataframe = pd.read_csv("https://raw.githubusercontent.com/rubengr16/MachineLearning/main/Prac1/test.csv", sep=",")
laptops_dataframe

,LaptopId,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight
0,891,HP,Envy x360,2 in 1 Convertible,13.3,Quad HD+ / Touchscreen 3200x1800,Intel Core i7 7500U 2.7GHz,16GB,256GB SSD,Intel HD Graphics 620,Windows 10,1.42kg
1,1149,Lenovo,ThinkPad X1,2 in 1 Convertible,14.0,IPS Panel Touchscreen 2560x1440,Intel Core i7 6500U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 10,1.27kg
2,962,Acer,Aspire F5-573G-510L,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,12GB,128GB SSD + 1TB HDD,Nvidia GeForce GTX 950M,Windows 10,2.4kg
3,1204,Asus,FX502VM-DM105T (i7-6700HQ/8GB/1TB/GeForce,Gaming,15.6,Full HD 1920x1080,Intel Core i7 6700HQ 2.6GHz,8GB,1TB HDD,Nvidia GeForce GTX 1060,Windows 10,2.2kg
4,1139,HP,EliteBook 850,Ultrabook,15.6,Full HD 1920x1080,Intel Core i7 6500U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 7,1.88kg
...,...,...,...,...,...,...,...,...,...,...,...,...
386,268,HP,Probook 470,Notebook,17.3,Full HD 1920x1080,Intel Core i7 8550U 1.8GHz,8GB,1TB HDD,Nvidia GeForce 930MX,Windows 10,2.5kg
387,1278,Dell,Inspiron 3552,Notebook,15.6,1366x768,Intel Celeron Dual Core N3050 1.6GHz,2GB,500GB HDD,Intel HD Graphics,Windows 10,2.20kg
388,299,Asus,ZenBook UX530UQ-PRO,Ultrabook,15.6,Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,16GB,512GB SSD,Nvidia GeForce 940MX,Windows 10,1.63kg
389,162,Dell,Inspiron 3576,Notebook,15.6,Full HD 1920x1080,Intel Core i5 8250U 1.6GHz,8GB,256GB SSD,AMD Radeon 520,Windows 10,2.13kg


Manipulamos o convertimos aquellas columnas para unificar o tenerlas en representación numérica:

In [62]:
unit = None
for i in range(len(laptops_dataframe)):
  unit = laptops_dataframe.loc[i,'Ram'][-2:].upper()
  if (unit == 'GB'):
    laptops_dataframe.loc[i,'Ram'] = int(laptops_dataframe.loc[i,'Ram'][:-2]) * 1024
  elif (unit == 'MB'):
    laptops_dataframe.loc[i,'Ram'] = int(laptops_dataframe.loc[i,'Ram'])

for i in range(len(laptops_dataframe)):
  laptops_dataframe.loc[i,'Weight'] = float(laptops_dataframe.loc[i,'Weight'][:-2])

laptops_dataframe


,LaptopId,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight
0,891,HP,Envy x360,2 in 1 Convertible,13.3,Quad HD+ / Touchscreen 3200x1800,Intel Core i7 7500U 2.7GHz,16384,256GB SSD,Intel HD Graphics 620,Windows 10,1.42
1,1149,Lenovo,ThinkPad X1,2 in 1 Convertible,14.0,IPS Panel Touchscreen 2560x1440,Intel Core i7 6500U 2.5GHz,8192,256GB SSD,Intel HD Graphics 520,Windows 10,1.27
2,962,Acer,Aspire F5-573G-510L,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,12288,128GB SSD + 1TB HDD,Nvidia GeForce GTX 950M,Windows 10,2.4
3,1204,Asus,FX502VM-DM105T (i7-6700HQ/8GB/1TB/GeForce,Gaming,15.6,Full HD 1920x1080,Intel Core i7 6700HQ 2.6GHz,8192,1TB HDD,Nvidia GeForce GTX 1060,Windows 10,2.2
4,1139,HP,EliteBook 850,Ultrabook,15.6,Full HD 1920x1080,Intel Core i7 6500U 2.5GHz,8192,256GB SSD,Intel HD Graphics 520,Windows 7,1.88
...,...,...,...,...,...,...,...,...,...,...,...,...
386,268,HP,Probook 470,Notebook,17.3,Full HD 1920x1080,Intel Core i7 8550U 1.8GHz,8192,1TB HDD,Nvidia GeForce 930MX,Windows 10,2.5
387,1278,Dell,Inspiron 3552,Notebook,15.6,1366x768,Intel Celeron Dual Core N3050 1.6GHz,2048,500GB HDD,Intel HD Graphics,Windows 10,2.2
388,299,Asus,ZenBook UX530UQ-PRO,Ultrabook,15.6,Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,16384,512GB SSD,Nvidia GeForce 940MX,Windows 10,1.63
389,162,Dell,Inspiron 3576,Notebook,15.6,Full HD 1920x1080,Intel Core i5 8250U 1.6GHz,8192,256GB SSD,AMD Radeon 520,Windows 10,2.13
